In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
with open('/kaggle/input/game-of-thrones/got1.txt') as f:
    data = f.read()

In [6]:
data[:100]

'A Game Of Thrones \nBook One of A Song of Ice and Fire \nBy George R. R. Martin \nPROLOGUE \n"We should '

In [7]:
from gensim.utils import simple_preprocess

In [8]:
data1 = simple_preprocess(data)

In [9]:
npdata1 = np.array(data1)
len(np.unique(npdata1)), len(npdata1)

(11307, 284700)

In [10]:
data2 = ' '.join(data1)

In [11]:
sequences_doc = []
seq_len = 50
l = seq_len + 1
tokens = [w for w in data2.split()]

for i in range(l, len(tokens)):
    
    seq = tokens[i-l:i]

    line = ' '.join(seq)
    sequences_doc.append(line)

In [12]:
sequences_doc[:10]

['game of thrones book one of song of ice and fire by george martin prologue we should start back gared urged as the woods began to grow dark around them the wildlings are dead do the dead frighten you ser waymar royce asked with just the hint of smile gared did',
 'of thrones book one of song of ice and fire by george martin prologue we should start back gared urged as the woods began to grow dark around them the wildlings are dead do the dead frighten you ser waymar royce asked with just the hint of smile gared did not',
 'thrones book one of song of ice and fire by george martin prologue we should start back gared urged as the woods began to grow dark around them the wildlings are dead do the dead frighten you ser waymar royce asked with just the hint of smile gared did not rise',
 'book one of song of ice and fire by george martin prologue we should start back gared urged as the woods began to grow dark around them the wildlings are dead do the dead frighten you ser waymar royce as

In [13]:
from keras.preprocessing.text import Tokenizer

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences_doc)
sequences = tokenizer.texts_to_sequences(sequences_doc)
vocab_size = len(tokenizer.word_index) + 1
vocab_size

11308

In [15]:
sequences = np.array(sequences)
sequences

array([[1098,    4, 1774, ...,  276, 1025,   60],
       [   4, 1774, 1302, ..., 1025,   60,   19],
       [1774, 1302,   46, ...,   60,   19,  909],
       ...,
       [   8,  192,    8, ...,   16,    1, 1699],
       [ 192,    8,   94, ...,    1, 1699,    4],
       [   8,   94, 3289, ..., 1699,    4,  737]])

In [16]:
X, y = sequences[:,:-1], sequences[:,-1]
X,y

(array([[1098,    4, 1774, ...,    4,  276, 1025],
        [   4, 1774, 1302, ...,  276, 1025,   60],
        [1774, 1302,   46, ..., 1025,   60,   19],
        ...,
        [   8,  192,    8, ...,  721,   16,    1],
        [ 192,    8,   94, ...,   16,    1, 1699],
        [   8,   94, 3289, ...,    1, 1699,    4]]),
 array([  60,   19,  909, ..., 1699,    4,  737]))

In [17]:
# from tensorflow.keras.utils import to_categorical

In [18]:
# y = to_categorical(y, num_classes=vocab_size)

In [19]:
seq_length = X.shape[1]
seq_length

50

In [20]:
y.shape

(284649,)

def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
    # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict(encoded, verbose=0)
        yhat = np.argmax(yhat,axis=1)
        print(yhat)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)# Model

In [58]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, dot, concatenate, Flatten

def define_model(vocab_size, seq_length):
    # Define input layer
    inputs = Input(shape=(seq_length,))
    
    # Embedding layer
    embedding_layer = Embedding(vocab_size, 50)(inputs)
    
    # First LSTM layer with return_sequences=True
    lstm_layer1 = LSTM(100, return_sequences=True)(embedding_layer)
    
    # Second LSTM layer with return_sequences=True
    lstm_layer2 = LSTM(100, return_sequences=True)(lstm_layer1)
    
    # Attention mechanism
    attention_scores = dot([lstm_layer2, lstm_layer1], axes=[2, 2])
    attention_scores = tf.nn.softmax(attention_scores, axis=-1)
    context = dot([attention_scores, lstm_layer1], axes=[2, 1])
    
    combined = concatenate([context, lstm_layer2])
    
    # Flatten the combined tensor
    flattened = Flatten()(combined)
    
    # Dense layers
    dense_layer1 = Dense(100, activation='relu')(flattened)
    output_layer = Dense(vocab_size, activation='sigmoid')(dense_layer1)  # Output shape is (None, 1)
    
    # Define the model
    model = Model(inputs=inputs, outputs=output_layer)
    
    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Summarize the model
    model.summary()
    
    return model

# Call the function to define the model
model = define_model(vocab_size, seq_length)


Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 embedding_10 (Embedding)       (None, 50, 50)       565400      ['input_10[0][0]']               
                                                                                                  
 lstm_18 (LSTM)                 (None, 50, 100)      60400       ['embedding_10[0][0]']           
                                                                                                  
 lstm_19 (LSTM)                 (None, 50, 100)      80400       ['lstm_18[0][0]']                
                                                                                            

In [59]:
from keras.utils.np_utils import to_categorical

In [53]:
from scipy.sparse import csr_matrix

In [54]:
# new_y = to_categorical(y)

In [60]:
y_binary = csr_matrix((np.ones(y.size), (np.arange(y.size), y)))

In [61]:
# y = to_categorical(y, num_classes=vocab_size)

In [62]:
model.fit(X, y, batch_size=2560, epochs=10)

Epoch 1/10
112/112 [==============================] - 27s 200ms/step - loss: 7.0199 - accuracy: 0.0571
Epoch 2/10
112/112 [==============================] - 19s 165ms/step - loss: 6.6917 - accuracy: 0.0622
Epoch 3/10
112/112 [==============================] - 14s 126ms/step - loss: 6.6852 - accuracy: 0.0622
Epoch 4/10
112/112 [==============================] - 14s 125ms/step - loss: 6.6790 - accuracy: 0.0622
Epoch 5/10
112/112 [==============================] - 14s 122ms/step - loss: 6.6715 - accuracy: 0.0622
Epoch 6/10
112/112 [==============================] - 13s 118ms/step - loss: 6.6249 - accuracy: 0.0622
Epoch 7/10
112/112 [==============================] - 12s 111ms/step - loss: 6.5666 - accuracy: 0.0622
Epoch 8/10
112/112 [==============================] - 13s 116ms/step - loss: 6.5141 - accuracy: 0.0623
Epoch 9/10
112/112 [==============================] - 13s 114ms/step - loss: 6.4634 - accuracy: 0.0627
Epoch 10/10
112/112 [==============================] - 12s 110ms/step - l

In [63]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
    # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict(encoded, verbose=0)
        yhat = np.argmax(yhat,axis=1)
        print(yhat)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [69]:
pip install keras-preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [70]:
from keras_preprocessing.sequence import pad_sequences

In [71]:
seed_text = sequences_doc[np.random.randint(0,len(sequences_doc))]
print(seed_text + '\n')
generate_seq(model, tokenizer, seq_length, seed_text, 50)

guiltily jory promised not to tell jory kept his word her father said with smile there are some things do not need to be told even blind man could see that wolf would never have left you willingly we had to throw rocks she said miserably told her to run to

[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]


'the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the'

In [72]:
seed_text = sequences_doc[np.random.randint(0,len(sequences_doc))]
print(seed_text + '\n')
generate_seq(model, tokenizer, seq_length, seed_text, 50)[:60]

and stars and not before page never the darkness cried never never never inside the tent dany found cushion soft silk stuffed with feathers she clutched it to her breasts as she walked back out to drogo to her sun and stars if look back am lost it hurt even to

[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]


'the the the the the the the the the the the the the the the '